In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["HF_KEY"]=os.getenv("HF_KEY")


In [2]:
# calling model
from langchain_groq import ChatGroq
model=ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.7)

In [3]:
# define the prompt
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate(
    [
        ("system" , "You are a teacher and you have to create {noofquestion} multiple choice question which have 4 option based on provide context {context} and set difficulties level according to the user {medium}")
    ]
)


Now to work on pdf to give the content to the model
process or step

pdf_load --> Data_ingesion --> splitting_data --> Embedding_data --> text to vector --> store vector --> chromadb

1.> pypdf , pymupdf 
2.> text_splitter , Recursivtext
3.> embedding by hugging face
4.> store in chromadb


In [4]:
from  langchain_community.document_loaders import PyPDFLoader
loaded_pdf=PyPDFLoader('deeplearning.pdf')

In [5]:
result_pdf=loaded_pdf.load()
print(result_pdf[0].page_content)

Introduction to Deep Learning
1. What is Deep Learning?
Deep Learning is a subset of Machine Learning, which itself is a subset of Artificial Intelligence (AI). 
It is inspired by the structure and function of the human brain, namely the artificial neural network. 
Deep learning algorithms attempt to learn high-level abstractions in data by using multiple layers of
processing.
Deep learning has driven significant advances in areas such as computer vision, speech
recognition, natural language processing (NLP), and autonomous vehicles. It is capable of handling
large volumes of data and finding complex patterns.
2. Difference Between AI, ML, and Deep Learning
- **Artificial Intelligence (AI)**: A broad field aimed at building smart machines capable of performing
tasks that typically require human intelligence.
- **Machine Learning (ML)**: A subset of AI that allows systems to learn and improve from
experience without being explicitly programmed.
- **Deep Learning (DL)**: A subset of ML t

In [6]:
# NOw break down loaded content into chunks 
from  langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=500 , chunk_overlap=10)

In [7]:
splitted_text=splitter.split_documents(result_pdf)

In [8]:
print(splitted_text)

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250708063420', 'source': 'deeplearning.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Introduction to Deep Learning\n1. What is Deep Learning?\nDeep Learning is a subset of Machine Learning, which itself is a subset of Artificial Intelligence (AI). \nIt is inspired by the structure and function of the human brain, namely the artificial neural network. \nDeep learning algorithms attempt to learn high-level abstractions in data by using multiple layers of\nprocessing.\nDeep learning has driven significant advances in areas such as computer vision, speech'), Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250708063420', 'source': 'deeplearning.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='recognition, natural language processing (NLP), and autonomous vehicles

In [9]:
puretext=result_pdf[0].page_content
print(puretext)

Introduction to Deep Learning
1. What is Deep Learning?
Deep Learning is a subset of Machine Learning, which itself is a subset of Artificial Intelligence (AI). 
It is inspired by the structure and function of the human brain, namely the artificial neural network. 
Deep learning algorithms attempt to learn high-level abstractions in data by using multiple layers of
processing.
Deep learning has driven significant advances in areas such as computer vision, speech
recognition, natural language processing (NLP), and autonomous vehicles. It is capable of handling
large volumes of data and finding complex patterns.
2. Difference Between AI, ML, and Deep Learning
- **Artificial Intelligence (AI)**: A broad field aimed at building smart machines capable of performing
tasks that typically require human intelligence.
- **Machine Learning (ML)**: A subset of AI that allows systems to learn and improve from
experience without being explicitly programmed.
- **Deep Learning (DL)**: A subset of ML t

In [10]:
splitted_text_pure=splitter.split_text(puretext)
print(splitted_text_pure)

['Introduction to Deep Learning\n1. What is Deep Learning?\nDeep Learning is a subset of Machine Learning, which itself is a subset of Artificial Intelligence (AI). \nIt is inspired by the structure and function of the human brain, namely the artificial neural network. \nDeep learning algorithms attempt to learn high-level abstractions in data by using multiple layers of\nprocessing.\nDeep learning has driven significant advances in areas such as computer vision, speech', 'recognition, natural language processing (NLP), and autonomous vehicles. It is capable of handling\nlarge volumes of data and finding complex patterns.\n2. Difference Between AI, ML, and Deep Learning\n- **Artificial Intelligence (AI)**: A broad field aimed at building smart machines capable of performing\ntasks that typically require human intelligence.\n- **Machine Learning (ML)**: A subset of AI that allows systems to learn and improve from\nexperience without being explicitly programmed.', '- **Deep Learning (DL)

In [11]:
# now embedding 
from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding=HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\Acer\AppData\Local\Temp\ipykernel_7648\3392400315.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")
d:\ollama_project\ollama_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
embeded_result=embedding.embed_documents(splitted_text_pure)
embeded_result

[[-0.06237494945526123,
  -0.034947920590639114,
  0.04355236887931824,
  -0.03551815450191498,
  -0.02579597570002079,
  -0.015728922560811043,
  -0.020195242017507553,
  -0.04580356925725937,
  -0.08395966142416,
  -0.07712394744157791,
  -0.09922454506158829,
  -0.023185838013887405,
  -0.023243039846420288,
  -0.027871545404195786,
  -0.06987506151199341,
  -0.043230198323726654,
  0.012099012732505798,
  0.021668406203389168,
  -0.14425519108772278,
  -0.05255040526390076,
  0.0728093609213829,
  0.045631859451532364,
  -0.03424272686243057,
  -0.0004210179322399199,
  0.016445893794298172,
  0.09730894863605499,
  0.06260789930820465,
  -0.026896242052316666,
  0.004483623430132866,
  0.008064055815339088,
  0.07285389304161072,
  -0.03131786361336708,
  0.053504351526498795,
  0.08263764530420303,
  -0.04612031951546669,
  0.07365170866250992,
  -0.07479138672351837,
  0.01734010875225067,
  0.012954470701515675,
  0.018622761592268944,
  -0.04870699718594551,
  -0.0013781579909

In [13]:
from langchain_community.vectorstores import Chroma
db_chroma=Chroma.from_texts(splitted_text_pure,embedding)

In [14]:
query_result=db_chroma.similarity_search("high-level abstractions in data by using multiple layers of\nprocessing")
print(query_result)

[Document(metadata={}, page_content='- **Deep Learning (DL)**: A subset of ML that uses neural networks with many layers (deep neural\nnetworks) to analyze various types of data.\n3. Neural Networks Basics\nArtificial Neural Networks (ANNs) are computing systems inspired by the biological neural networks.\nA neural network consists of layers of nodes, or "neurons". Each connection between neurons has a\nweight and a bias.\n- **Input Layer**: Takes in the features of the data.'), Document(metadata={}, page_content='Introduction to Deep Learning\n1. What is Deep Learning?\nDeep Learning is a subset of Machine Learning, which itself is a subset of Artificial Intelligence (AI). \nIt is inspired by the structure and function of the human brain, namely the artificial neural network. \nDeep learning algorithms attempt to learn high-level abstractions in data by using multiple layers of\nprocessing.\nDeep learning has driven significant advances in areas such as computer vision, speech'), Docu

In [15]:
# Now we will bring this in output form 
from langchain_core.output_parsers import StrOutputParser
output=StrOutputParser()


In [16]:
# now ask the query

chain=prompt|model|output
result=chain.invoke({"noofquestion" : "10" , "context" :query_result , "medium" :"easy"})

In [17]:
print(result)

Here are 10 multiple-choice questions based on the provided context, with 4 options each, and difficulty levels set to "easy":

1. What is Deep Learning a subset of?
A) Artificial Intelligence (AI)
B) Machine Learning (ML)
C) Natural Language Processing (NLP)
D) Computer Vision

Answer: B) Machine Learning (ML)
Difficulty: Easy

2. What is the main inspiration for Deep Learning?
A) Human brain structure and function
B) Computer vision
C) Natural Language Processing (NLP)
D) Autonomous vehicles

Answer: A) Human brain structure and function
Difficulty: Easy

3. What type of networks are used in Deep Learning?
A) Shallow neural networks
B) Deep neural networks
C) Artificial neural networks
D) Biological neural networks

Answer: B) Deep neural networks
Difficulty: Easy

4. What is the primary function of the Input Layer in a neural network?
A) To process the data
B) To analyze the data
C) To take in the features of the data
D) To output the results

Answer: C) To take in the features of t

In [18]:
chain=prompt|model|output
result=chain.invoke({"noofquestion" : "10" , "context" :query_result , "medium" :"Hard"})

In [19]:
print(result)

Here are 10 multiple-choice questions based on the provided context, with 4 options each, and difficulty level set to Hard:

1. What is the primary inspiration for Deep Learning algorithms?
A) Biological neural networks
B) Artificial Intelligence
C) Machine Learning
D) Computer Vision

Answer: A) Biological neural networks

2. Which of the following is a characteristic of Deep Learning?
A) It uses a single layer of processing
B) It is a subset of Artificial Intelligence
C) It is incapable of handling large volumes of data
D) It uses multiple layers of processing to learn high-level abstractions

Answer: D) It uses multiple layers of processing to learn high-level abstractions

3. What is the function of the Input Layer in a Neural Network?
A) To process the output of the network
B) To take in the features of the data
C) To learn high-level abstractions
D) To handle large volumes of data

Answer: B) To take in the features of the data

4. Which of the following areas has been significan